In [2]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [15]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_year/"

if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)
    
### IS THIS RACHAEL'S DATASET OR SOMEONE WHO HAS VIDEOINFO.TXT IN THEIR FOLDER???

all_folders_drop, excel_drop, txt_drop = get_dropbox_video_folders("/DATA/MYRISTATE/DATA/1_year/", True)

# folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "452"]
# all_folders_drop = pd.read_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

clear_output(wait=False)

all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230329_Plate083_01/Img/,083,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate083/0...,01/Img/
1,20230329_Plate083_02/Img/,083,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate083/0...,02/Img/
2,20230329_Plate083_03/Img/,083,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate083/0...,03/Img/
3,20230329_Plate057_02/Img/,057,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate057/0...,02/Img/
4,20230329_Plate057_03/Img/,057,20230329,DATA/MYRISTATE/DATA/1_year/20230329_Plate057/0...,03/Img/
...,...,...,...,...,...
279,20230512_Plate118_008/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,008/Img/
280,20230512_Plate118_009/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,009/Img/
281,20230512_Plate118_010/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,010/Img/
282,20230512_Plate118_011/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,011/Img/


In [16]:
# directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_dataset/"
# all_folders_drop.to_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# save_dropbox_state("/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate552/", is_video=True)

folders_drop = all_folders_drop
# folders_drop = folders_drop.loc[folders_drop["video"] == "084"]

In [17]:
folders_drop = folders_drop.sort_values(by=['folder'], ignore_index=True)
folders_drop["folder"] = [f"{entry.split('_')[0]}_{entry.split('_')[1]}/{entry.split('_')[2]}" for entry in folders_drop["folder"]]

In [18]:
# FINAL CHECK

print(directory_targ)
print(folders_drop["tot_path_drop"].iloc[0])
folders_drop

/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_year/
DATA/MYRISTATE/DATA/1_year/20230321_Plate058/01/Img


,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230321_Plate058/01/Img/,058,20230321,DATA/MYRISTATE/DATA/1_year/20230321_Plate058/0...,01/Img/
1,20230321_Plate058/02/Img/,058,20230321,DATA/MYRISTATE/DATA/1_year/20230321_Plate058/0...,02/Img/
2,20230321_Plate098/01/Img/,098,20230321,DATA/MYRISTATE/DATA/1_year/20230321_Plate098/0...,01/Img/
3,20230321_Plate098/02/Img/,098,20230321,DATA/MYRISTATE/DATA/1_year/20230321_Plate098/0...,02/Img/
4,20230321_Plate098/03/Img/,098,20230321,DATA/MYRISTATE/DATA/1_year/20230321_Plate098/0...,03/Img/
...,...,...,...,...,...
279,20230512_Plate118/008/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,008/Img/
280,20230512_Plate118/009/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,009/Img/
281,20230512_Plate118/010/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,010/Img/
282,20230512_Plate118/011/Img/,118,20230512,DATA/MYRISTATE/DATA/1_year/20230512_Plate118/0...,011/Img/


In [19]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "10:00:00",
    "transfer_test"
)


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2926787


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2926788
Submitted batch job 2926789


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 2927457
Submitted batch job 2927459


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2927460
Submitted batch job 2927461


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2927462


In [21]:
# print(excel_drop)
for xl_adress in excel_drop:
    file_name = f"{xl_adress.split('/')[-2]}/{xl_adress.split('/')[-1]}"
    if file_name.split(".")[-2][-4:] == 'data':
        continue
    print(file_name.split(".")[-2][-4:])

    download(xl_adress, directory_targ+file_name)

te98
2023-06-16 13:44:15,922-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 13:44:16,116-[INFO]- dropbox:474 -> Request to files/download
e110
2023-06-16 13:44:16,517-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 13:44:16,665-[INFO]- dropbox:474 -> Request to files/download
e058
2023-06-16 13:44:17,009-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 13:44:17,157-[INFO]- dropbox:474 -> Request to files/download
e116
2023-06-16 13:44:17,641-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 13:44:17,805-[INFO]- dropbox:474 -> Request to files/download
e041
2023-06-16 13:44:18,408-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 13:44:18,605-[INFO]- dropbox:474 -> Request to files/download
e055
2023-06-16 13:44:19,090-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 13:44:19,241-[INFO]- dropbox:474 -> Request to files/download
e062
2023-06-16 13:44:19,862-[INFO]- dropbox:390 -> Refreshing access token.
2023-06-16 13:44:20,023